In [115]:
import json
import re
from datetime import datetime
import concurrent.futures
import pandas as pd

In [2]:
with open('../linux-commits-2023-11-12.json') as f:
    all_commits = f.readlines()

In [39]:
all_commits_dict_list = []
all_commits_map = {}
for commit in all_commits:
    commit_dict = json.loads(commit)

    # Just a fair idea about multiple-key
    for n in [5, 6, 7, 8, 9, 10, 11, 12]:
        # To cover collisions
        if commit_dict['data']['commit'][0:n] in all_commits_map:
            all_commits_map[commit_dict['data']['commit'][0:n]].append(commit_dict)
        else:
            all_commits_map[commit_dict['data']['commit'][0:n]] = [commit_dict]
    all_commits_dict_list.append(commit_dict)

In [42]:
date_format = '%a %b %d %H:%M:%S %Y %z'
def datesDistance(bfc_date, bic_date):
    bfc_date_formated = datetime.strptime(bfc_date, date_format)
    bic_date_formated = datetime.strptime(bic_date, date_format)
    return bfc_date_formated - bic_date_formated

In [87]:
def analyzeCornerCases(commit):
    global http_links

    commit_message = commit['data']['message'].replace('"','').replace('(',' ').replace(')',' ')#.lower()
    match = re.search(r"Fixes:[^\S\n]+(\w+)", commit_message)
    if match is not None:
        bic_hash = match.group(1)
    else:
        return None
        
    # There are fixes with a link to Bugzilla/Other sites
    if bic_hash.startswith("http"):
        http_links+=1
        return None

    if bic_hash in [
        'IRQ','NB','SLI','Bug','line','tag',
        'tags','Discovery','discovery','drivers','igt','Bugzilla','bugzilla',
        'correctly','computation','terminate','Configure','addresses',
        'hashes', 'second'
    ]:
        return None

    if bic_hash == "commit":
        match = re.search(r"Fixes:[^\S\n]+commit[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        bic_hash = match.group(1)

    elif bic_hash == "Commit":
        match = re.search(r"Fixes:[^\S\n]+Commit[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        bic_hash = match.group(1)

    # Double 'Fixes: ' (7 cases)
    elif bic_hash == "Fixes":
        match = re.search("Fixes: Fixes:[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        if match is not None:
            bic_hash = match.group(1) 

    # Special format (3 cases)
    elif bic_hash == "linux":
        match = re.search("Fixes: linux-next commit[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit_message)
        if match is not None:
            bic_hash = match.group(1)

    # 77 cases, no commits hashes (manually checked)
    elif len(bic_hash[0:12]) < 6:
        return None

    return bic_hash

In [108]:
def analyzeCommit(commit):
    global errors
    global not_in_git
    global not_sha
    global perfect_samples
    global short_commit_hash
    global corner_cases
    
    try:
        # There are commits without message
        if 'message' not in commit['data']:
            return None

        bic_hash = ""

        # Search: "Fixes: <hash>"
        match = re.search(r"Fixes:[^\S\n]+(\b[0-9a-f]{5,40}\b)", commit['data']['message'])
        if match is not None:
            bic_hash = match.group(1)
            perfect_samples+=1
        else:
            #return None
            # Search: "Fixes <something>"
            bic_hash = analyzeCornerCases(commit)
            if bic_hash is None: return None
            corner_cases+=1

        # Last check, if commit hash not in map, discard it
        if bic_hash[0:12] not in all_commits_map:
            not_in_git+=1
            return None
                
        candidates = all_commits_map[bic_hash[0:12]]

        if len(candidates) > 1:
            print("Collision", bic_hash[0:12], candidates[0]['data']['commit'],candidates[1]['data']['commit'], "BFC: ", commit['data']['commit'])
            return None
        else:
            bic = candidates[0]
        
        delta = datesDistance(commit['data']['CommitDate'],bic['data']['CommitDate'])
        result = {
            'BFC_hash': commit['data']['commit'],
            'BIC_hash':  bic['data']['commit'],
            'BFC_comment': commit['data']['message'].split('\n', 1)[0],
            'BIC_comment': bic['data']['message'].split('\n', 1)[0],
            'daysDistance': delta.days
        }
        return result

    except Exception as e:
        errors+=1
        match = re.search("Fixes: (.*)", commit['data']['message'])    
        print("Error matching: ",match.group(0), "BFC:",commit['data']['commit'])
        return None

In [109]:
links = []
errors = 0
corner_cases = 0
http_links = 0
not_in_git = 0
perfect_samples = 0
for commit in all_commits_dict_list:
    result = analyzeCommit(commit)
    if result is not None:
        links.append(result)
print("http_links",http_links)
print("not_in_git",not_in_git)
print("perfect_samples",perfect_samples)
print("corner_cases",corner_cases)
print("errors",errors)

Collision 22a5dc 22a5dc0e5e3e8fef804230cd73ed7b0afd4c7bae 22a5dc10e3f8fb8370748ea19dc4e3e1620d8296 BFC:  d7924450e14ea414568563ec01489f77452b00b4
Collision 4f1982 4f198289747f0391bc5a5574279b1791a8ca2d06 4f1982b4e262c45475a91b4253e9bc7f7c991c13 BFC:  3fd61b209977db8a9fe6c44d5a5a7aee7a255f64
Error matching:  Fixes: commit BFC: 7ea38c6c3622bc65279dc6a1fecd28227027fbb5
Error matching:  Fixes: commit 8700e3e7c48A5 ("Add Soft RoCE driver") BFC: e259934d4df7f99f2a5c2c4f074f6a55bd4b1722
http_links 139
not_in_git 1163
perfect_samples 91959
corner_cases 422
errors 2


In [110]:
len(links)

91216

In [120]:
filtered_links = []
for link in links:
    if link['BIC_hash'] != '1da177e4c3f41524e886b7f1b8a0c1fc7321cac2':
        filtered_links.append(link)
len(filtered_links)

90760

In [122]:
df = pd.DataFrame(filtered_links)
df.to_csv('links.csv', index=False)  

456

## Rare cases
- No commit message: https://github.com/torvalds/linux/commit/7b7abfe3dd81d659a0889f88965168f7eef8c5c6
- No exist in repository (but in GitHub): https://github.com/torvalds/linux/commit/54fe26a900bc528f3df1e4235cb6b9ca5c6d4dc2
- No exist (even in GitHub): 21d2202158e9